In [ ]:
import ollama
def generate(prompt):
    response = ollama.chat(model='gemma3:4b-it-qat', messages=[
    {
            "role": "system",
            "content": """You are an expert programmer helping debug Python errors.

    You are given a traceback, which may include system/library/internal calls. Your task is to:
    - Only return the lines relevant to debugging the error (i.e., exclude system, standard library, and internal framework calls).
    - Keep lines that are inside the user's project code or directly contributed to the crash.
    - Keep the final error message.

    Only return these important lines — no explanations or extra commentary."""
        },
        {
            "role": "user",
            "content": prompt
        }
    ])
    return response["message"]["content"]



In [7]:
from cohere import ClientV2

client = ClientV2(api_key="pDTQMeCH5sZRr9VV2v16Bpv0q2dhUIdQajmw3ptQ")        

def generate(prompt):
    response = client.chat(
	model="command-a-03-2025",
	messages=[
		{
			"role": "system",
			"content": "You are an expert programmer helping debug Python errors.\n\n	You are given a traceback, which may include system/library/internal calls. Your task is to:\n	- Only return the lines relevant to debugging the error (i.e., exclude system, standard library, and internal framework calls).\n	- Keep lines that are inside the user's project code or directly contributed to the crash.\n	- Keep the final error message.\n\n	Only return these important lines — no explanations or extra commentary."
		},
		{
			"role": "user",
			"content": [
				{
					"type": "text",
					"text": prompt
				}
			]
		}
	],
	temperature=1
)
    return response.message.content[0].text


In [6]:
print(generate("""Traceback (most recent call last):
File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
return _run_code(code, main_globals, None,
File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
exec(code, run_globals)
File "main.py", line 12, in <module>
my_function()
File "utils/math.py", line 44, in my_function
return 1 / 0
ZeroDivisionError: division by zero"""))

File "main.py", line 12, in <module>
my_function()
File "utils/math.py", line 44, in my_function
return 1 / 0
ZeroDivisionError: division by zero
None


In [8]:
import json
from scripts.bugsinpy_utils import *

projects = get_projects()
output_path = "filtered_tracebacks.json"

with open(output_path, "w") as f:
    f.write("[\n")
    first = True

    for project in projects:
        bugs = get_bugs(project)
        for bug in bugs:
            changed_files = parse_changed_files(project, bug)
            if len(changed_files) > 1:
                continue

            raw_error = get_raw_traceback(project, bug)
            if len(raw_error) > 10000:
                raw_error = raw_error[:10000]

            filtered_error = generate(raw_error)
            print(f"Project: {project}, Bug: {bug}. Before: {len(raw_error)} After: {len(filtered_error)}")

            entry = {
                "project": project,
                "bug_id": bug,
                "filtered_traceback": filtered_error
            }

            if not first:
                f.write(",\n")
            else:
                first = False

            json.dump(entry, f)

    f.write("\n]\n")


Project: matplotlib, Bug: 29. Before: 4205 After: 1895
Project: matplotlib, Bug: 14. Before: 3142 After: 1059
Project: matplotlib, Bug: 12. Before: 6512 After: 2906
Project: matplotlib, Bug: 6. Before: 4347 After: 2144
Project: matplotlib, Bug: 9. Before: 3449 After: 1240
Project: matplotlib, Bug: 22. Before: 7758 After: 4160
Project: matplotlib, Bug: 3. Before: 3041 After: 728
Project: matplotlib, Bug: 10. Before: 3557 After: 1125
Project: matplotlib, Bug: 7. Before: 7950 After: 574
Project: matplotlib, Bug: 21. Before: 3192 After: 838
Project: matplotlib, Bug: 13. Before: 3189 After: 798
Project: matplotlib, Bug: 16. Before: 9097 After: 3171
Project: matplotlib, Bug: 2. Before: 2996 After: 751
Project: matplotlib, Bug: 8. Before: 10000 After: 5072
Project: matplotlib, Bug: 18. Before: 5756 After: 881
Project: matplotlib, Bug: 19. Before: 3221 After: 964
Project: matplotlib, Bug: 23. Before: 3558 After: 1108
Project: matplotlib, Bug: 25. Before: 3499 After: 471
Project: matplotlib, Bu